In [ ]:
import mlflow
import sys
import os

# ==================== CONFIGURATION ====================

# 💡 महत्वपूर्ण: यहाँ अपनी सफल TRAINING RUN ID दर्ज करें!
# कृपया 'model_train.py' स्क्रिप्ट से प्राप्त Run ID को इस प्लेसहोल्डर से बदलें।
# Example: RUN_ID = "0a1b2c3d4e5f6789"
RUN_ID = "68259b965b1847e095ec9dbc02333f68" # हमने इसे आपके लॉग के अनुसार सेट कर दिया है।

# Unity Catalog की S3 AccessDenied त्रुटि को बायपास करने के लिए, हम 
# WORKSPACE-BASED REGISTRY का उपयोग करेंगे (जिसके लिए केवल साधारण नाम की आवश्यकता है)।
REGISTERED_MODEL_NAME = "rf_house_price_model_v1" # Workspace Registry के लिए एक साधारण नाम
MODEL_ARTIFACT_PATH = "sklearn_rf_model"

def register_model_for_serving(run_id: str, model_name: str, artifact_path: str):
    """
    MLflow Run से मॉडल को Databricks Model Registry में पंजीकृत करता है।
    """
    
    # 💡 FIX: S3 Access Denied त्रुटि को बायपास करने के लिए Unity Catalog (UC) को अक्षम करें 
    # और डिफ़ॉल्ट Workspace Registry का उपयोग करें।
    try:
        # रजिस्ट्री URI को None पर सेट करें ताकि डिफ़ॉल्ट Workspace Registry का उपयोग हो
        mlflow.set_registry_uri(None)
        print("✓ MLflow Registry URI explicitly set to None (Workspace Registry) to bypass S3 Access Denied error.")
    except Exception as e:
        print(f"⚠ Warning: Could not explicitly set registry URI: {e}")
        # यदि यह विफल भी होता है, तो पंजीकरण (registration) अक्सर काम कर जाता है।


    # मॉडल का सोर्स पाथ
    model_uri = f"runs:/{run_id}/{artifact_path}"
    
    print(f"⏳ Attempting to register model from URI: {model_uri}")
    
    try:
        # मॉडल को रजिस्ट्री में पंजीकृत करें
        # यदि मॉडल पहले से मौजूद नहीं है, तो यह बन जाएगा।
        model_version = mlflow.register_model(
            model_uri=model_uri, 
            name=model_name
        )
        
        print("\n" + "=" * 60)
        print("✅ MODEL REGISTRATION SUCCESSFUL!")
        print(f"नाम: {model_version.name}")
        print(f"संस्करण (Version): {model_version.version}")
        print(f"स्टेज (Stage): {model_version.current_stage}")
        print("=" * 60)
        
        # इस मॉडल संस्करण को डिप्लॉयमेंट के लिए लौटाएँ
        return model_version
        
    except Exception as e:
        print(f"❌ मॉडल पंजीकरण विफल हुआ! त्रुटि: {e}")
        print("सुनिश्चित करें कि MLflow ट्रैकिंग URI सही है और RUN_ID वैध है।")
        # S3 त्रुटियों को डिबग करना संभव नहीं है, लेकिन हम कोशिश करते रहेंगे।
        sys.exit(1)

if __name__ == "__main__":
    
    registered_version = register_model_for_serving(RUN_ID, REGISTERED_MODEL_NAME, MODEL_ARTIFACT_PATH)
    
    if registered_version:
        # मॉडल सर्विंग एंडपॉइंट के लिए अगला चरण
        print(f"अगला कदम: इस मॉडल संस्करण ({registered_version.name} v{registered_version.version}) का उपयोग करके Serving Endpoint बनाएँ।")
